### Process

A process is an instance of a computer program being executed. Each process has its own memory space it uses to store the instructions being run, as well as any data it needs to store and access to execute.

A process's memory can further be divided into the heap & the stack. The stack contains local variables, such as a variable declared and used inside a function. Data on the stack gets allocated & de-allocated as it moves in & out of scope. The heap can be used to create data that exists outside of a function with dynamic allocation using methods such as malloc or new. Within Python, the interpreter takes care of the majority of memory management.

### Threads
Threads are components of a process that can run in parallel. There can be multiple threads in a process, and they share the same memory space, i.e., the parent process's memory space.

This would mean the code to be executed, and variables declared in the program's heap can be shared by all threads, but variables on the stack will remain separate across threads.

Sharing variables across threads is a powerful aspect of programming but one that can cause various issues if not handled correctly. While threads run in parallel we have limited control on the ordering of code executing.
This can lead to scenarios where two threads attempt to manage the usage of a single variable leading to unexpected behavior!

### The GIL

The GIL (Global Interpreter Lock) ensures that there is no more than one thread in a state of execution at any given moment with the CPython interpreter. This lock is necessary mainly because CPython's memory management is not thread-safe. This results in the CPython interpreter being unable to do two things simultaneously.

However, we can achieve true parallelism & bypass the limitations of the GIL using multiprocessing. With multiprocessing, multiple Python processes can be created (resulting in multiple GILs, i.e., 1 per process) to perform parallel processing using multiple CPU cores.

More details on on the GIL: https://realpython.com/python-gil/

This program will be used as a baseline to compare its runtime with or without multi-threading or multi-processing.

In [4]:
import os
import math
import time

def calc():
   for i in range(0, 4000000):
       math.sqrt(i)

time_0 =  time.time()
for i in range(os.cpu_count()):
    calc()
time_1 =  time.time()
print(f'Execution time:{time_1 -time_0} seconds')

Execution time:5.171923637390137 seconds


### Muli-Threading
We can perform threading via the threading module. An example is shown below. This example is based on calculating the square root of 4 million numbers. The function to perform this is assigned to a thread. For the context of the example, a thread is created for every core within the system.

When you run this you will see that only a single Python process is run, even though multiple threads (based on the number of cores you have) are executed. This is because of the GIL.


In [1]:
from threading import Thread
import os
import math
import time

def calc():
   for i in range(0, 4000000):
       math.sqrt(i)

threads = list()

for i in range(os.cpu_count()):
   print('registering thread %d' % i)
   threads.append(Thread(target=calc))

time_0 =  time.time()
for thread in threads:
   thread.start()

for thread in threads:
   thread.join()
time_1 =  time.time()
print(f'Execution time:{time_1 -time_0} seconds')

registering thread 0
registering thread 1
registering thread 2
registering thread 3
registering thread 4
registering thread 5
registering thread 6
registering thread 7
registering thread 8
registering thread 9
registering thread 10
registering thread 11
registering thread 12
registering thread 13
registering thread 14
registering thread 15
Execution time:5.136295557022095 seconds


### Multiple Processing
In order to use multiple processing the multiprocessing module is used. As you can see this module is consumed is much the same as way the threading module`.

When run, you will see that multiple Python processes are created, one per core. Each one running its own GIL, and the parallel execution of calc() being performed.


In [2]:
from multiprocessing import Process
import os
import math
import time

def calc():
   for i in range(0, 4000000):
       math.sqrt(i)

processes = list()

for i in range(os.cpu_count()):
   print('registering process %d' % i)
   processes.append(Process(target=calc))

time_0 =  time.time()
for process in processes:
   process.start()

for process in processes:
   process.join()
time_1 =  time.time()
print(f'Execution time:{time_1 -time_0} seconds')

registering process 0
registering process 1
registering process 2
registering process 3
registering process 4
registering process 5
registering process 6
registering process 7
registering process 8
registering process 9
registering process 10
registering process 11
registering process 12
registering process 13
registering process 14
registering process 15
Execution time:0.42549967765808105 seconds


### So when should you use threading over multiprocessing?
To summarize you would typically want to use threading when your operations are I/O(Input/Output) bound.

For example, let's say making 20 API requests. As we know the GIL would prevent 20 parallel threads from running.

The GIL, as we know, will only allow a single thread to execute. However, in the example the running thread will reach out to the network for it's I/O operation.

As this I/O is performed outside of Python the GIL would release the lock, and allow the other threads to run. When the first thread’s I/O returned the lock would then be reacquired.

Therefore, threading has provided us with some additional benefit without requiring the overhead needed in creating multiple processes.

Of course, this is not good for executions that require greater computation, as the GIL/lock upon the thread would remain. In this case, multiprocessing is beneficial, allowing you to split your workload across multiple CPU cores.
